**This program demonstrates performs network characterization on wikipedia Vote Dataset**

In [0]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from networkx.algorithms import community
from networkx.algorithms import components
from networkx.algorithms.community import girvan_newman
import operator
import itertools

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls '/content/drive/My Drive/CMPE_ 256_Graph_ Assignment'

Slashdot0811.txt   soc-sign-bitcoinotc.csv  Wiki-Vote.txt
soc-Epinions1.txt  WikigraphAnalysis.ipynb


**Dataset wiki-Vote.txt is read in the dataframe**

In [3]:
df = pd.read_csv('/content/drive/My Drive/CMPE_ 256_Graph_ Assignment/Wiki-Vote.txt')
df.head(10)

,# Directed graph (each unordered pair of nodes is saved once): Wiki-Vote.txt
0,# Wikipedia voting on promotion to administrat...
1,# Nodes: 7115 Edges: 103689
2,# FromNodeId\tToNodeId
3,30\t1412
4,30\t3352
5,30\t5254
6,30\t5543
7,30\t7478
8,3\t28
9,3\t30


**Rows 0,1,2 are dropped from the dataframe**

In [0]:
df.drop([0,1,2],inplace=True) #drops first 3 rows

In [0]:
df.head(10)

,# Directed graph (each unordered pair of nodes is saved once): Wiki-Vote.txt
3,30\t1412
4,30\t3352
5,30\t5254
6,30\t5543
7,30\t7478
8,3\t28
9,3\t30
10,3\t39
11,3\t54
12,3\t108


**Reset index of the dataframe to 0 after dropping three rows**

In [0]:
df.reset_index(inplace=True,drop=True)

In [0]:
df.head()

,# Directed graph (each unordered pair of nodes is saved once): Wiki-Vote.txt
0,30\t1412
1,30\t3352
2,30\t5254
3,30\t5543
4,30\t7478


In [0]:
df.shape

(103689, 1)

**Split the column 0 into two columns named as Source_node and Target_node**

In [0]:
df = pd.DataFrame(df[df.columns[0]].str.split('\t',1).tolist(),        
                                   columns = ['Source_node','Target_node']) #split column 0 two columns Source_node and Target_node

In [0]:
df.head()

,Source_node,Target_node
0,30,1412
1,30,3352
2,30,5254
3,30,5543
4,30,7478


**A directed Graph H is created from the dataframe**

In [0]:
H = nx.from_pandas_edgelist(df, source='Source_node', target='Target_node', create_using=nx.DiGraph()) #create a directed graph from dataframe df

In [0]:
nx.is_directed(H)

True

In [0]:
print("Node Degree")
for v in H:
    print('%s %s' % (v, H.degree(v)))

Node Degree
30 28
1412 29
3352 537
5254 298
5543 282
7478 92
3 54
28 255
39 69
54 47
108 13
152 72
178 98
182 111
214 175
271 192
286 19
300 22
348 56
349 16
371 141
567 28
581 12
584 19
586 23
590 40
604 21
611 25
8283 10
25 90
6 322
8 226
19 61
23 96
29 191
33 58
35 175
50 46
55 131
75 100
80 102
86 230
94 107
127 151
154 79
192 46
224 31
226 46
246 24
252 38
255 112
257 33
261 32
273 20
280 98
282 140
299 158
302 37
306 266
308 183
310 273
322 43
325 70
339 100
356 61
358 12
363 21
370 45
373 20
377 28
379 21
407 280
415 70
422 16
428 22
439 78
488 26
514 20
545 73
549 66
560 48
592 29
600 291
645 24
650 112
659 58
664 39
667 54
673 47
675 26
696 39
706 119
723 26
739 60
756 64
857 115
864 20
893 53
959 90
994 78
1131 34
3755 121
8282 11
8284 10
4 29
10 101
38 28
56 173
130 33
140 52
147 73
151 111
232 45
243 86
341 60
606 18
5 23
61 20
89 45
93 18
153 19
163 23
167 12
168 33
175 20
34 25
132 57
135 42
171 123
183 27
216 29
219 18
227 32
228 50
236 60
250 22
258 19
259 134
274 29
28

**Prints number of nodes in the graph**

In [0]:
H.number_of_nodes() #prints number of nodes in the graph

7115

**Prints number of edges in the graph**

In [0]:
H.number_of_edges() #prints number of edges in the graph

103689

**Converted directed graph into undirected one**

In [8]:
H_undirected = nx.Graph(H) #converted graph H into undirected graph named as H_undirected
nx.is_directed(H_undirected)

False

**Checking if the undirected graph is connected or not**

In [0]:
print(nx.is_connected(H_undirected)) #checking if the undirected graph is connected or not

False


**Prints number of connected components in the undirected graph**

In [0]:
nx.number_connected_components(H_undirected) #prints number of connected components in the undirected graph

24

**Finds the diameter of undirected graph.**
**Since the graph is not connected, the diameter is infinity.So, First largest connected Component is found and then diameter of that component is calculated.**

In [0]:
Gc = max(nx.connected_component_subgraphs(H_undirected), key=len) #gives only largest connected component
nx.diameter(Gc) #prints the diameter of the each connected subgraph

7

**Prints average clustering coefficient of the undirected graph**

In [0]:
nx.average_clustering(H_undirected) #prints average clustering coefficient of the undirected graph

0.14089784589308738

**Calculates closeness centrality of the undirected graph and prints top five nodes with the highest closeness centrality**

In [0]:
cc = nx.closeness_centrality(H_undirected) #prints five nodes with the highest closeness centrality
sorted(cc.items(), key=lambda x: x[1], reverse=True)[:5]

[('2565', 0.48741490125142045),
 ('766', 0.46691538587304166),
 ('457', 0.46660487487625174),
 ('1549', 0.46586133082226927),
 ('1166', 0.4656758149276032)]

**Calculates betweenness centrality of the undirected graph and prints top five nodes with the highest betweenness centrality**

In [0]:
bc = nx.betweenness_centrality(H)
sorted(bc.items(), key=lambda x: x[1], reverse=True)[:5] # prints top 5 nodes with highest betweenness centrality

[('2565', 0.017654409558147836),
 ('1549', 0.016564095998753692),
 ('15', 0.01156258726064681),
 ('72', 0.008011822532712367),
 ('737', 0.006134997021063534)]

**Calculates Page Rank of the undirected graph with damping factor 0.85**

In [0]:
pr = nx.pagerank(H_undirected,alpha=0.85) # Page Rank of the directed graph    
print("Node PageRank")
print('%s' % pr)

Node PageRank
{'30': 0.0001519420295970706, '1412': 0.00027009254747362144, '3352': 0.0019525879232285515, '5254': 0.001420883364095316, '5543': 0.0011601173401803375, '7478': 0.0005475256334928842, '3': 0.0002553264053400368, '28': 0.0013206264022200284, '39': 0.00037782528519348654, '54': 0.0002396030366645042, '108': 0.00010589231201963163, '152': 0.00041106746997705536, '178': 0.00044856068330143055, '182': 0.0004715537347278227, '214': 0.0010717293236942457, '271': 0.0009286325801151901, '286': 9.947562011920545e-05, '300': 0.0001281002801454951, '348': 0.000259094694421394, '349': 9.122964848007005e-05, '371': 0.0005852805485038237, '567': 0.0002260313240958663, '581': 7.754809696180836e-05, '584': 0.00010131740024872271, '586': 0.00011940315851059695, '590': 0.0001821208488053907, '604': 0.00012559092792704935, '611': 0.0001437822512124007, '8283': 7.820901519667703e-05, '25': 0.000413529090703603, '6': 0.0014071396248192722, '8': 0.001056689972482109, '19': 0.000264982081036058

**Find the top two nodes with highest page rank**

In [0]:
sorted_pr = sorted(pr.items(), key=operator.itemgetter(1),reverse=True) #sorts the page ranks of the nodes
print(sorted_pr[:2]) #find the top two nodes with highest page rank

[('2565', 0.00432884820637688), ('11', 0.00301243768596226)]


**Finds the dispersion between two nodes with highest pageRank**

In [0]:
nx.algorithms.centrality.dispersion(H,'2565','11') #prints the dispersion between two nodes with highest pageRank

73.49549549549549

**Finds hubs and authorities of the directed graph and prints top 5 nodes with the highest authority**

In [0]:
hubs,auth=nx.hits(H, max_iter=100, tol=1e-08, nstart=None, normalized=True) #finds hubs and authorities of the directed graph H
sorted_auth = sorted(auth.items(), key=operator.itemgetter(1),reverse=True) #sort the authorities
print(sorted_auth[:5]) #print 5 nodes with highest authority score

[('2398', 0.002580147178008918), ('4037', 0.002573241124142803), ('3352', 0.002328415091537902), ('1549', 0.0023037314804751075), ('762', 0.00225587485637424)]


**Finds the communities using girvan newman algorithm for a directed graph**

In [0]:
comp = girvan_newman(H)

**Printing number of total communities**

In [0]:
sum=0
for c in comp:
  sum=sum+1
print("Number of communities using girvan newman are :",sum)

KeyboardInterrupt: ignored

**printing 5 nodes from the top 5 communities**

In [0]:
k=2
for communities in itertools.islice(comp,k):
  for c in communities:
    sorted_communities = sorted(c)
    print(tuple(sorted_communities[:5]))
    #print((tuple(sorted_communities = sorted(c) for c in communities))[:5])

In [0]:
sum(1 for x in comp) #Printing number of total communities

0

In [0]:
import itertools
k = 5
comp = girvan_newman(H)
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities)[:5])  

(['10', '100', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '101', '1010', '1011', '1012', '1013', '1014', '1016', '1017', '1018', '1019', '102', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1028', '1029', '103', '1030', '1031', '1032', '1033', '1034', '1035', '1037', '1038', '1039', '104', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '105', '1050', '1051', '1052', '1053', '1055', '1058', '1059', '106', '1060', '1061', '1062', '1063', '1064', '1065', '1066', '1067', '1068', '107', '1070', '1071', '1072', '1074', '1075', '1077', '1078', '1079', '108', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '109', '1090', '1092', '1093', '1094', '1095', '1096', '1097', '1098', '1099', '11', '110', '1100', '1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108', '1109', '111', '1110', '1111', '1112', '1113', '1114', '1115', '112', '1120', '1123', '1124', '1125', '1126',

KeyboardInterrupt: ignored